In [1]:
import os
import sys
import numpy as np
import networkx as nx
import scanpy as sc
import matplotlib.pyplot as plt


## Read publibhed patient SCLC data

The dataset is published through the following paper:

```
Chan JM, Quintanal-Villalonga Á, Gao VR, Xie Y, Allaj V, Chaudhary O, Masilionis I, Egger J, Chow A, Walle T, Mattar M. Signatures of plasticity, metastasis, and immunosuppression in an atlas of human small cell lung cancer. Cancer Cell. 2021 Nov 8;39(11):1479-96.
```

See [here](https://cellxgene.cziscience.com/collections/62e8f058-9c37-48bc-9200-e767f318a8ec) and [here](https://data.humantumoratlas.org/explore?selectedFilters=%5B%7B%22group%22%3A%22AtlasName%22%2C%22value%22%3A%22HTAN+MSK%22%7D%2C%7B%22value%22%3A%22hdf5%22%2C%22label%22%3A%22hdf5%22%2C%22group%22%3A%22FileFormat%22%2C%22count%22%3A11%2C%22isSelected%22%3Afalse%7D%5D&tab=file#) for the files.


In [2]:
def filter_rna(rna, nCells=-1, min_genes=100, min_cells=3, min_counts=3, n_top_genes=2000, pct_mito=40):
    # 1. Filter mitochondrial genes
    rna.var['mt'] = rna.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
    plt.hist(rna.obs["pct_counts_mt"], bins=100)
    rna = rna[rna.obs["pct_counts_mt"] < pct_mito]
    # 2. Filter cells with too few genes
    sc.pp.filter_cells(rna, min_genes=min_genes)
    # 3. Filter genes with too few counts, or expressed in too few cells
    sc.pp.filter_genes(rna, min_counts=min_counts) 
    sc.pp.filter_genes(rna, min_cells=min_cells) 
    if nCells > -1:
        sc.pp.subsample(rna,  n_obs=nCells)
    # Find highly variable genes and only keep them
    # Stuart, Tim, et al. "Comprehensive integration of single-cell data." Cell 177.7 (2019): 1888-1902.
    # 1. Using count data, compute mean and variance of each gene
    # 2. Fit a loess with span .3 to variance against mean plot (function sigma(mean_j) for gene j)
    # 3. Compute the regularized z_score as z_ij = (x_ij - mean_j) / sigma(mean_j), then clip by sqrt(N) for N the number fo cells
    # 4. Rank by regularized z_score
    sc.pp.highly_variable_genes(rna, n_top_genes=n_top_genes, flavor="seurat_v3", subset=True)
    rna.layers["counts"] = rna.X.copy()
    # Normalize, log transform and scale (changes .X)
    # Normalize each cell by the total counts in that cell, multiplying by median total counts per cell (so all cells have same total counts)
    sc.pp.normalize_total(rna)
    sc.pp.log1p(rna)
    return(rna)


def preprocess_rna(rnaDir, rna=None, nCells=-1, **kwargs):
    """
    Preprocess the RNA data
    Load, handle genomic coordinates, normalize and scale, compute PCA
    Only keeps higly variable genes (based on seurat v3 definition)
    Remove genes with unknown genomic coordinates
    input:
        rnaDir: directory containing the RNA data
        nCells: number of cells to keep (if -1, keep all)

    output:
        rna: anndata.AnnData object
    """
    if rna is None:
        rna = sc.read_10x_mtx(rnaDir)
    
    # Find gene coordinates & remove genes with no coordinates
    # Minimal filtering...
    rna = filter_rna(rna=rna, nCells=nCells, **kwargs)
    return(rna)


In [ ]:
outDir = "/path/to/data/"
adata = sc.read(os.path.join(outDir, "Ru1322b_6634_4000.h5ad"))

# count the number of cells per 'sample' and sort 
adata.obs['sample'].value_counts().sort_values(ascending=False)

adata.obs['sample'].unique()
bdata = adata[adata.obs['sample'].str.contains('Ru1322b')].copy()

# Lets keep 4K genes, and all the cells
bdata.X = bdata.layers['counts'].copy()
b1 = preprocess_rna(rnaDir=None, rna=bdata, gtfPath=None, nCells=-1, min_genes=100, min_cells=3, min_counts=3, n_top_genes=4000, pct_mito=80)
b1.obs['labels'] = np.random.choice(['A', 'B', 'C'], size=b1.shape[0])
outDir = '/juno/work/shah/users/salehis/projects/rnaseq-pfm/data/sclc_pub/'
os.makedirs(outDir, exist_ok=True)
os.makedirs(os.path.join(outDir, 'processed'), exist_ok=True)
b1.write(os.path.join(outDir, 'Ru1322b_6634_4000.h5ad'))

In [ ]:
# create the train/validation/test split
#!./driver.py setup_data -i ../data/sclc_pub/Ru1322b_6634_4000.h5ad -o ../data/sclc_pub/processed -p .2 -f True -l 1